In [137]:
import os
import time
import h5py
import numpy as np
import math
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn.functional as F
import random
def generate_generators(data_root, hdf_data_path,BZ,IR,
                        label_scheme='label'):
    """
    Gathers all files and organize into train/validation/test. Each
       data subset is further organized by class. Converts file lists
       into iterable data generators.
    Inputs:
        * data_root: full path to the data root containing subdirectories of data
        * trn_list, val_list, tst_list: each is a list of strings. The strings
            should encode the strata being used to divide data into segments and
            must match the corresponding field of the FileName class.
        * hdf_data_path: the path used internally in the HDF to get the desired surface
        * batch_size: the _approximate_ number of samples to be used __next__ call. Note
              that if batch_size // n_classes is non-integer then it may be slightly off.
        * label_scheme: a string that matches the name of the property of the FileName
              class that you want to use as the label.
        * strata_scheme: a string that matches the name of the property of the FileName
              class that you want to compare against trn_list (etc.) for data segmentation.
    """
    n_classes = 2
    tst_files = [[] for clas in range(n_classes)]
    trn_files = [[] for clas in range(n_classes)]

    
    #print('Allocating HDFs to train/valid/test...')
    for filename in os.listdir(data_root):
        if not filename.endswith('.hdf'):
            continue

        # Extract the label using 'label_scheme' identifier
        label = int(int(filename.split('_')[3]) > 0)  # 0 = clutter (not manmade); 1 = target (manmade)

        # list that stores files in two sub-lists
        randr = random.randrange(0,5)
        if randr < 1:
            
            tst_files[label].append(filename)

        else:

            trn_files[label].append(filename)

    # Wrap each file list into an iterable data generator that actually
    #     read the HDFs when __next__ is called:
    
    trn_gen = DataGenerator(data_root, trn_files, hdf_data_path,  BZ, IR)
    tst_gen = DataGenerator(data_root, tst_files, hdf_data_path, BZ, IR)
    length = len(trn_files[0] + trn_files[1]) #+ len(trn_files[1])

    return trn_gen, tst_gen


class DataGenerator:
    def __init__(self, data_root, file_list, hdf_data_path, BZ, IR, n_classes=2):
        # Basic properties:
        self.data_root = data_root
        self.n_classes = n_classes

        "balance by class"
        self.hdf_path = hdf_data_path

        self._permanent_file_list = file_list
        # print(len(file_list[0]), "\n")
        "for august-november split"
        # self.target_list = file_list[1][0:4
        "for fulll dataset"
        self.target_list = file_list[1]
        self.target_len = len(self.target_list)
        # print(self.target_len, 'length of target')
        
        "test"

        #self.clutter_list = file_list[0]
        self.clutter_list = file_list[0][0:int(IR*self.target_len)]

        #self.clutter_list = file_list[0][0:16799]
        self.clutter_len = len(self.clutter_list)
        # print(self.clutter_len, 'length of clutter')

        self.dataset_size = self.clutter_len + self.target_len
        self.batch_size = BZ
        self.bsz_by_class = int(self.batch_size / self.n_classes)

        if self.batch_size % 2 != 0 or self.bsz_by_class % 2 != 0:
            print('batch size is odd or not balanced... \nadding one to batch size')
            self.batch_size = self.batch_size  + 1
            self.bsz_by_class = math.floor(self.batch_size / self.n_classes)

        n = 1
        while n < 6:
            if  self.bsz_by_class % 2 != 0:
                print('batch size is not balanced... \nadding one to batch size')
                self.batch_size = self.batch_size  + 1
                self.bsz_by_class = math.floor(self.batch_size / self.n_classes)
            else:
                break



        # print(self.batch_size, 'batch_size')
        # print(self.bsz_by_class, 'balance size')

        #self.current_index = [i for i in range(self.batch_size)]


        #self.current_index = [0 for label in range(self.n_classes)]

        self.HDF_n_rows = 71
        self.HDF_n_cols = 71
        self.HDF_n_dpth = 101

        self.input_shape = (self.batch_size, self.HDF_n_rows, self.HDF_n_cols, self.HDF_n_dpth)

        self.chip_n_rows = 64
        self.chip_n_cols = 64
        self.chip_n_dpth = 101

        "chip shape is one cube from batch"
        self.chip_shape = (self.chip_n_rows, self.chip_n_cols, self.chip_n_dpth)

        self.batch_shape = (self.batch_size, self.chip_n_rows, self.chip_n_cols, self.chip_n_dpth)


        # Make a boolean indexing mask for efficient extraction of the chip center:
        "input shape minus chip shape row"
        row_diff = self.input_shape[1] - self.chip_shape[0]
        "this is chopping off one half the difference on each side"
        first_row = row_diff // 2
        last_row = first_row + self.chip_shape[0]

        "this is chopping off one half the difference on each side"
        col_diff = self.input_shape[2] - self.chip_shape[1]
        first_col = col_diff // 2
        last_col = first_col + self.chip_shape[1]

        "this is chopping off one half the difference on each side"
        slice_diff = self.input_shape[3] - self.chip_shape[2]
        first_slice = slice_diff // 2
        last_slice = first_slice + self.chip_shape[2]

        "slicing out the chips from the input data"
        "list with shape of input all False values"
        self.center_select = np.full((self.HDF_n_rows, self.HDF_n_cols, self.HDF_n_dpth), False)
        "except the chips size all true"
        self.center_select[first_row:last_row, first_col:last_col, first_slice:last_slice] = True

        # self.first_slice = 0
        # self.last_slice = self.bsz_by_class


    def readHDF(self, file_name):
        """ Reads data from the HDF"""
        with h5py.File(os.path.join(self.data_root, file_name), mode='r') as f:
            data = f[self.hdf_path][:]
            data = np.transpose(data)  # because python reverses the order of the 3d volume, this will correct back to the original matlab order
            #data = data / 40  # Now data is in [0,1], to match the 2d sensors


        return data

    def chip_center(self, data):
        """ extracts the center of the chip via boolean indexing. """
        return np.reshape(data[self.center_select], self.chip_shape)


    def preprocess(self, data_sample):
        """
        Preprocesses a data sample.
        TODO: use configuration file and preprocesser class like ADAM dataloader here.
        """
        # TODO: sometimes want to jiggle the chip instead of centering so will
        #          need to remove chip_center from here.

        "centering x cube"
        x_center = self.chip_center(data_sample)
        return x_center

    def perm_target_list(self, target_list):
        if self.last_slice % self.target_len == 0:
            print('target list permuted')
            return np.random.permutation(target_list)
        else:
            return target_list
        #return np.random.permutation(target_list)

    def reset_list(self, target_list):

        # Come up with a way to reset_list


            return target_list

    def data_loop(self, list_data):

        "place holder for batch of data"
        batch_data = np.zeros(self.batch_shape, dtype='float32')
        "batch labels"
        batch_label = np.zeros(self.batch_size)

        for label in range(0, self.n_classes):
            for nth_sample in range(0, self.bsz_by_class):

                ld = list_data[label]
                sample = ld[nth_sample]

                data = self.readHDF(sample)

                # Preprocessing can go here, e.g. random flips/translations/normalizations
                "centers the data here"
                data = self.preprocess(data)

                # Insert the data into the batch_data and batch_label arrays:
                batch_idx = self.bsz_by_class * label + nth_sample
                "batch data: why just rows and columns"
                batch_data[batch_idx][:, :, :] = np.reshape(data, self.chip_shape)
                "batch label"
                batch_label[batch_idx] = label

        return batch_data, batch_label

    def __iter__(self):
        self.first_slice = 0
        self.last_slice = self.bsz_by_class
        return self

    def __len__(self):
        return sum([len(label_set) for label_set in self.target_list])

    def __next__(self):


        # Put `bsz_by_class' samples from each class into `batch_data':

        # for n in range(self.bsz_by_class, self.target_len, self.bsz_by_class):


        if self.last_slice <= self.clutter_len:
            #print(self.last_slice, 'last slice')

            if self.last_slice > self.target_len:
                tlp = self.perm_target_list(self.target_list)
                tl = self.reset_list(tlp)

                cl = self.clutter_list[self.first_slice:self.last_slice]

                list_data = [cl, tl]

                batch_data, batch_label = self.data_loop(list_data)

            elif self.last_slice <= self.target_len :
                tl = self.target_list[self.first_slice:self.last_slice]
                cl = self.clutter_list[self.first_slice:self.last_slice]


                list_data = [cl, tl]


                batch_data, batch_label = self.data_loop(list_data)

        else:
            print('next epoch')
            raise StopIteration

        self.first_slice += 1
        self.last_slice += 1
        return batch_data, batch_label
        # , self.bsz_by_class, self.clutter_len, self.target_len, self.batch_size




In [138]:
def relu(x):
    _relu = nn.ReLU()
    
    return _relu(x)
# change to global min / max
def curly_N(w):
    w_min, w_max = torch.min(torch.min(torch.min(w))), torch.max(torch.max(torch.max(w)))
    reg_N = (w - w_min) / (w_max - w_min)
    return reg_N

def curly_Nprime(w):
    w_min, w_max = torch.min(torch.min(torch.min(w))), torch.max(torch.max(torch.max(w)))
    curly_N = (w - w_min + 1) / (w_max - w_min + 2)
    return curly_N
    # return (w - torch.min(w) + 1) / (torch.max(w) - torch.min(w) + 2)

def f_VHN(x, w):
    relu_x = relu(curly_N(x))
    relu_w = relu(curly_Nprime(w))
    
    return relu_x * relu_w
    
class VHNLayer(nn.Module):
    """ Custom VHN layer """
    def __init__(self, channels, img_len, img_width):
        super().__init__()
        self.channels, self.img_len, self.img_width = channels, img_len, img_width
        weights = torch.Tensor(channels, img_len, img_width)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        
        

    def forward(self, x):
        
        return f_VHN(x, self.weights) 

In [139]:

# from load_data import device
"fix dimensions"
"may need "
import matplotlib.pyplot as plt
# from colormap import *
import os

class ATR(nn.Module):
    def __init__(self, nc, device = None, N_filters=4, N_output = 1, ker = 4, s = 2, pad =1):
        super(ATR, self).__init__()
        self.ker = ker
        self.s = s
        self.pad = pad
        self.nc = nc
        # self.device = device
        self.N_filters = N_filters
        self.N_output = N_output
        self.vhn = VHNLayer(101, 64, 64)
        self.conv1 = nn.Conv3d(nc, N_filters, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.conv2 = nn.Conv3d(N_filters, N_filters, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.conv3 = nn.Conv3d(N_filters, N_filters, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.conv4 = nn.Conv3d(N_filters, 1, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.avgpool = nn.AvgPool3d(kernel_size = (6, 2, 2), stride= (1, 1, 1), padding= (0, 0, 0))
    

        # "columns input x and output columnes"

        self.f2 = nn.Linear(1248,1)

        self.sigmoid = nn.Sigmoid()


    def forward(self, x0):
        "image vectorization"
        # print(x0.shape)
        x0 = self.vhn.forward(x0)
        x = self.conv1(x0.unsqueeze(1))
        x = self.avgpool(F.relu(x))
        x = self.conv2(x)
        x = self.avgpool(F.relu(x))
        x = self.conv3(x)
        x = self.avgpool(F.relu(x))
        x = self.conv4(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = self.f2(x)

        y = self.sigmoid(x)

        return y

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
 

In [140]:
net = ATR(1)

In [141]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [142]:
print(f"Using {device} device")

Using cuda device


In [143]:
path = '../../../research_data/sas_full_data/'

In [144]:
dldr_trn, dldr_tst = generate_generators(data_root = path , hdf_data_path = 'DL_info/chip_info/cube_raw',BZ = 20, IR = 1)

In [145]:
import torch.optim as optim

criterion1 = nn.BCELoss()
criterion2 = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(2):  # loop over the dataset multiple times
    print("epoch {epoch}".format(epoch=epoch))
    running_loss = 0.0
    for i, data in enumerate(dldr, 0):
        # get the inputs; data is a list of [inputs, labels]
    
        inputs, labels = data
        
        inputs = torch.log10(torch.tensor(inputs).transpose(1,3) + 1)
        # print(inputs.shape)
        labels = torch.tensor(labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # print(inputs)
        # forward + backward + optimize
        outputs = net(inputs)
        loss1 = criterion1(outputs, labels.reshape(20,1).type(torch.float32))
        
        loss2 = criterion2(curly_Nprime(net.vhn.weights), curly_N(torch.sum(inputs, dim = 0) / dldr.batch_size))
        # print("netvhn", net.vhn.weights.shape)
        # print(curly_N(torch.sum(inputs, dim = 0) / dldr.batch_size).shape)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        if i % 5 == 4:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5:.3f}')
            running_loss = 0.0

epoch 0
[1,     5] loss: 0.735
[1,    10] loss: 0.735
[1,    15] loss: 0.732
[1,    20] loss: 0.736
[1,    25] loss: 0.732
[1,    30] loss: 0.725
[1,    35] loss: 0.720
[1,    40] loss: 0.718
[1,    45] loss: 0.714
[1,    50] loss: 0.717
[1,    55] loss: 0.704
[1,    60] loss: 0.704
[1,    65] loss: 0.679
[1,    70] loss: 0.629
[1,    75] loss: 0.577
[1,    80] loss: 0.589
[1,    85] loss: 0.568
[1,    90] loss: 0.460
[1,    95] loss: 0.487
[1,   100] loss: 0.493
[1,   105] loss: 0.339
[1,   110] loss: 0.299
[1,   115] loss: 0.419
[1,   120] loss: 0.569
[1,   125] loss: 0.561
[1,   130] loss: 0.507
[1,   135] loss: 0.423
[1,   140] loss: 0.348
[1,   145] loss: 0.401
[1,   150] loss: 0.530
[1,   155] loss: 0.407
[1,   160] loss: 0.244
[1,   165] loss: 0.273
[1,   170] loss: 0.389
[1,   175] loss: 0.528
[1,   180] loss: 0.481
[1,   185] loss: 0.308
[1,   190] loss: 0.351
[1,   195] loss: 0.504
[1,   200] loss: 0.601
[1,   205] loss: 0.606
[1,   210] loss: 0.474
[1,   215] loss: 0.530
[1,

In [146]:
preds = []
labels = []


import torcheval
from torcheval.metrics.functional import binary_auprc
with torch.no_grad():
    for i, data in enumerate(dldr_tst,0):
        inputs, label = data
        
        inputs = torch.log10(torch.tensor(inputs).transpose(1,3) + 1)
        # print(inputs.shape)
        label = torch.tensor(label)
        # calculate outputs by running images through the 
        output = net(inputs)

        preds.append(output.tolist())
        labels.append(label.tolist())


        if i > 100:
            break
print(len(preds))
print("PRAUC", binary_auprc(torch.tensor(preds).squeeze(0).squeeze(2), torch.tensor(labels), num_tasks=len(preds)).mean())

next epoch
81
PRAUC tensor(0.8908)


In [147]:
class ATR(nn.Module):
    def __init__(self, nc, device = None, N_filters=4, N_output = 1, ker = 4, s = 2, pad =1):
        super(ATR, self).__init__()
        self.ker = ker
        self.s = s
        self.pad = pad
        self.nc = nc
        # self.device = device
        self.N_filters = N_filters
        self.N_output = N_output
        self.conv1 = nn.Conv3d(nc, N_filters, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.conv2 = nn.Conv3d(N_filters, N_filters, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.conv3 = nn.Conv3d(N_filters, N_filters, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.conv4 = nn.Conv3d(N_filters, 1, kernel_size = (3, 3, 3), stride=(1, 2, 2), padding= (0, 1, 1))
        self.avgpool = nn.AvgPool3d(kernel_size = (6, 2, 2), stride= (1, 1, 1), padding= (0, 0, 0))
    

        # "columns input x and output columnes"

        self.f2 = nn.Linear(1248,1)

        self.sigmoid = nn.Sigmoid()


    def forward(self, x0):
        "image vectorization"
        # print(x0.shape)
        x = self.conv1(x0.unsqueeze(1))
        x = self.avgpool(F.relu(x))
        x = self.conv2(x)
        x = self.avgpool(F.relu(x))
        x = self.conv3(x)
        x = self.avgpool(F.relu(x))
        x = self.conv4(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = self.f2(x)

        y = self.sigmoid(x)

        return y

In [148]:
import torch.optim as optim

criterion1 = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(2):  # loop over the dataset multiple times
    print("epoch {epoch}".format(epoch=epoch))
    running_loss = 0.0
    for i, data in enumerate(dldr, 0):
        # get the inputs; data is a list of [inputs, labels]
    
        inputs, labels = data
        
        inputs = torch.log10(torch.tensor(inputs).transpose(1,3) + 1)
        # print(inputs.shape)
        labels = torch.tensor(labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # print(inputs)
        # forward + backward + optimize
        outputs = net(inputs)
        loss1 = criterion1(outputs, labels.reshape(20,1).type(torch.float32))
        # print("netvhn", net.vhn.weights.shape)
        # print(curly_N(torch.sum(inputs, dim = 0) / dldr.batch_size).shape)
        loss = loss1
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        if i % 5 == 4:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5:.3f}')
            running_loss = 0.0

epoch 0
[1,     5] loss: 0.441
[1,    10] loss: 0.603
[1,    15] loss: 0.529
[1,    20] loss: 0.414
[1,    25] loss: 0.354
[1,    30] loss: 0.383
[1,    35] loss: 0.373
[1,    40] loss: 0.288
[1,    45] loss: 0.373
[1,    50] loss: 0.365
[1,    55] loss: 0.292
[1,    60] loss: 0.327
[1,    65] loss: 0.280
[1,    70] loss: 0.345
[1,    75] loss: 0.335
[1,    80] loss: 0.413
[1,    85] loss: 0.434
[1,    90] loss: 0.308
[1,    95] loss: 0.451
[1,   100] loss: 0.549
[1,   105] loss: 0.336
[1,   110] loss: 0.177
[1,   115] loss: 0.313
[1,   120] loss: 0.564
[1,   125] loss: 0.519
[1,   130] loss: 0.422
[1,   135] loss: 0.342
[1,   140] loss: 0.263
[1,   145] loss: 0.300
[1,   150] loss: 0.432
[1,   155] loss: 0.303
[1,   160] loss: 0.154
[1,   165] loss: 0.184
[1,   170] loss: 0.274
[1,   175] loss: 0.430
[1,   180] loss: 0.336
[1,   185] loss: 0.217
[1,   190] loss: 0.286
[1,   195] loss: 0.430
[1,   200] loss: 0.543
[1,   205] loss: 0.538
[1,   210] loss: 0.382
[1,   215] loss: 0.407
[1,

In [149]:
preds = []
labels = []


import torcheval
from torcheval.metrics.functional import binary_auprc
with torch.no_grad():
    for i, data in enumerate(dldr_tst,0):
        inputs, label = data
        
        inputs = torch.log10(torch.tensor(inputs).transpose(1,3) + 1)
        # print(inputs.shape)
        label = torch.tensor(label)
        # calculate outputs by running images through the network
        output = net(inputs)
        # the class with the highest energy is what we choose as prediction
        preds.append(output.tolist())
        labels.append(label.tolist())
        print(i)

        if i > 100:
            break
print(len(preds))
print("PRAUC", binary_auprc(torch.tensor(preds).squeeze(0).squeeze(2), torch.tensor(labels), num_tasks=len(preds)).mean())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
next epoch
81
PRAUC tensor(0.9048)
